In [22]:
"""
Manual transaction simplification system

The algorithm:
1. Take the sum of all debts (+ or -) for all people
    - These are eveyone's centralized debts to the "central node"
2. Merge the central node with the designated payee (transfer everyone's
pairwise debts to this payee); this will be zero sum
3. Use a greedy algorithm to assign people who are net in debt to people
who will net receive money

This gives dumb results in some cases. If A owes 1000 people 1 cent and B
owes A 10 dollars, than this will make B owe 1000 people 1 cent.
"""

def transaction_adjacency_graph(pairwise_transactions):
    """
    pairwise_transaction:
      List of hashes [{"debtor": "person_who_owes",
                       "pay_to": "name",
                       "amount": int},
                      ...]
      amount must be positive
    returns:
      Undirected weighted adjacency list of debts
      {"person_name_a": {"person_name_b": amount, ...}, ... }
    """
    # get all the people first and set up empty hashes for them all
    all_people = ( {t["debtor"] for t in pairwise_transactions} |
                   {t["pay_to"] for t in pairwise_transactions} )
    adjacency_graph = {k: {} for k in all_people}
    # fill the graph
    for transaction in pairwise_transactions:
        debtor = transaction["debtor"]
        pay_to = transaction["pay_to"]
        amount = transaction["amount"]
        adjacency_graph[debtor][pay_to] = -amount
        adjacency_graph[pay_to][debtor] = amount
    return adjacency_graph

def simplify_transactions(adjacency_graph):
    """
    Accepts a graph encoded as an undirected weighted adjacency list
      {"person_name_a": {"person_name_b": amount, ...}, ... }

    Returns a hash of {"person_name_a": {"person_name_b": amount}}
      where person_name_a pays amount to person_name_b
      amount is always positive
    """
    people = adjacency_graph.keys()
    people_to_total_debt = {k: sum(adjacency_graph[k].values())
                            for k in people}
    receivers = [[p, people_to_total_debt[p]] for p in people
                  if people_to_total_debt[p] > 0]
    givers = [[p, -people_to_total_debt[p]] for p in people
               if people_to_total_debt[p] < 0]

    # assign debts
    simplified_debt_hash = {}
    current_receiver = 0
    current_giver = 0
    while current_receiver < len(receivers):
        if givers[current_giver][1] < receivers[current_receiver][1]:
            give_amount  = givers[current_giver][1]
            give_name    = givers[current_giver][0]
            receive_name = receivers[current_receiver][0]
            givers[current_giver][1]       -= give_amount
            receivers[current_receiver][1] -= give_amount
            simplified_debt_hash[give_name] = {receive_name: give_amount}
            current_giver += 1
        else:
            give_amount  = receivers[current_receiver][1]
            give_name    = givers[current_giver][0]
            receive_name = receivers[current_receiver][0]
            givers[current_giver][1]       -= give_amount
            receivers[current_receiver][1] -= give_amount
            simplified_debt_hash[give_name] = {receive_name: give_amount}
            current_receiver += 1
    return simplified_debt_hash

In [23]:
test_case_adjacency_graph = transaction_adjacency_graph([{"debtor": "Kevin", "pay_to": "Xander", "amount": 100}, {"debtor": "Nick", "pay_to": "Xander", "amount": 100}, {"debtor": "Kevin", "pay_to": "Nick", "amount": 30}])

In [24]:
simplify_transactions(test_case_adjacency_graph)

{'Nick': {'Xander': 70}, 'Kevin': {'Xander': 130}}